<a href="https://colab.research.google.com/github/prasad1482/AI-Resume-Matcher-Smart-Resume-to-Job-Description-Matching-with-BERT/blob/main/preprocessing_AI_resume_matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/expanded_job_resume_dataset.csv")

# Display first few rows
print(df.head())


            Job Title                                Job Skills Required  \
0      Data Scientist  Python, Machine Learning, Pandas, TensorFlow, SQL   
1   Marketing Manager  SEO, Google Ads, Content Marketing, Brand Stra...   
2  Frontend Developer                JavaScript, React, CSS, HTML, Redux   
3   Marketing Manager  SEO, Google Ads, Content Marketing, Brand Stra...   
4  Frontend Developer                JavaScript, React, CSS, HTML, Redux   

  Candidate Name                                   Candidate Skills  \
0          Frank  Recruitment, Employee Relations, Payroll, HR A...   
1            Bob  Python, Machine Learning, Pandas, TensorFlow, SQL   
2            Eve  SEO, Google Ads, Content Marketing, Brand Stra...   
3            Bob  Python, Machine Learning, Pandas, TensorFlow, SQL   
4            Bob  Python, Machine Learning, Pandas, TensorFlow, SQL   

   Match Label  
0            0  
1            0  
2            0  
3            0  
4            0  


#Install necessary libraries


In [3]:
!pip install nltk
!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

#clean dataset


In [7]:
print(df.columns)


Index(['Job Title', 'Job Skills Required', 'Candidate Name',
       'Candidate Skills', 'Match Label'],
      dtype='object')


In [12]:
# Function to clean text
def clean_text(text):
    if pd.isnull(text):
        return ""  # Handle missing values
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(words)

# Apply cleaning function
df["cleaned_resume"] = df["Candidate Skills"].apply(clean_text)
df["cleaned_job_desc"] = df["Job Skills Required"].apply(clean_text)

# Display cleaned dataset
display(df[["cleaned_resume", "cleaned_job_desc"]].head())

# Save cleaned dataset
df.to_csv("cleaned_dataset.csv", index=False)


,cleaned_resume,cleaned_job_desc
0,recruitment employee relations payroll hr anal...,python machine learning pandas tensorflow sql
1,python machine learning pandas tensorflow sql,seo google ads content marketing brand strategy
2,seo google ads content marketing brand strategy,javascript react css html redux
3,python machine learning pandas tensorflow sql,seo google ads content marketing brand strategy
4,python machine learning pandas tensorflow sql,javascript react css html redux


#Feature extraction


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Combine the cleaned skills for vectorization
job_desc_vectors = vectorizer.fit_transform(df["cleaned_job_desc"])
resume_vectors = vectorizer.transform(df["cleaned_resume"])

# Compute similarity scores
similarity_scores = []
for i in range(len(df)):
    score = cosine_similarity(resume_vectors[i], job_desc_vectors[i])
    similarity_scores.append(score[0][0])

# Add similarity scores to the DataFrame
df["match_score"] = similarity_scores

# Display the updated dataset
df[["Candidate Name", "Job Title", "match_score"]].head()


,Candidate Name,Job Title,match_score
0,Frank,Data Scientist,0.0
1,Bob,Marketing Manager,0.0
2,Eve,Frontend Developer,0.0
3,Bob,Marketing Manager,0.0
4,Bob,Frontend Developer,0.0


#model training

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Create the binary classification label
# You can adjust the threshold for a good match, here I set it to 0.6
df["Match Label"] = (df["match_score"] > 0.6).astype(int)

# Split the data into training and testing sets
X = df[["match_score"]]  # Features (match scores)
y = df["Match Label"]    # Labels (good/bad match)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest Classifier (or you can use Logistic Regression)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       252
           1       1.00      1.00      1.00        48

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



#save the model

In [15]:
import joblib

# Save the trained model to a file
joblib.dump(model, 'resume_job_match_model.pkl')


['resume_job_match_model.pkl']